# Baseline models

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import LSTM, Convolution1D, GlobalAveragePooling1D, Dense, Dropout, Masking, TimeDistributed
import keras_tuner as kt

from lifelines import KaplanMeierFitter
from lifelines.utils import median_survival_times
from lifelines import CoxPHFitter
from lifelines.statistics import proportional_hazard_test

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
import pydot

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import random
import torchinfo

# seed = 35
# random.seed(seed)
# np.random.seed(seed)
# tf.random.set_seed(seed)

# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.9.2


In [3]:
train_data_full_df = pd.read_csv('../../data_analysis/fd002-scaled_train_data.csv', sep=' ')
test_data_df = pd.read_csv('../../data_analysis/fd002-scaled_test_data.csv', sep=' ')

train_labels_full_df = pd.read_csv('../../data_analysis/fd002-training_labels.csv', sep=' ')
test_labels_df = pd.read_csv('../../data_analysis/fd002-testing_labels.csv', sep=' ')
test_labels_at_break_df = pd.read_csv('../../TED/CMAPSSData/RUL_FD002.txt', sep=' ', header=None)

FileNotFoundError: [Errno 2] No such file or directory: '../../data_analysis/scaled_train_data.csv'

In [ ]:
test_labels_df

## Kaplan Meier

### Adding event_observed column: 0 for not-observed (test_data), 1 for observed (train_data)

In [ ]:
km_train = train_data_full_df.copy()
km_test = test_data_df.copy()
km_train['Event Observed'] = 1
km_test['Event Observed'] = 0

km_total = pd.concat([km_train, km_test])

In [ ]:
kmf = KaplanMeierFitter()
kmf.fit(durations = km_total['Cycle'], event_observed = km_total['Event Observed'])
kmf.plot_survival_function()

In [ ]:
median_ = kmf.median_survival_time_
median_confidence_interval_ = median_survival_times(kmf.confidence_interval_)
print(median_)
print(median_confidence_interval_)

## Cox Proportional Hazard

In [ ]:
cox_train = train_data_full_df.copy()
cox_test = test_data_df.copy()
cox_train['Event Observed'] = 1
cox_test['Event Observed'] = 0

cox_total = pd.concat([cox_train, cox_test])

cox_total.drop(columns=['ID', 'OpSet1', 'OpSet2', 'OpSet3', 'SensorMeasure1', 'SensorMeasure5', 'SensorMeasure18', 'SensorMeasure19'], inplace=True)

In [ ]:
cox_total

In [ ]:
cph = CoxPHFitter()
cph.fit(cox_total, duration_col = 'Cycle', event_col = 'Event Observed')
cph.print_summary()

In [ ]:
plt.subplots(figsize = (10, 6))
cph.plot()

## Data pre-processing

In [ ]:
train_full_df = train_data_full_df.copy()
test_df = test_data_df.copy()
train_labels_full_df = train_labels_full_df.copy().clip(upper=125)
test_labels_df = test_labels_df.copy()

used_sensors = []
used_sensors.append("ID")
used_sensors.append("Cycle")
for i in range(1, 22):
    if i not in [1, 5, 6, 10, 16, 18, 19]:
        used_sensors.append("SensorMeasure" + str(i))

train_full_df = train_full_df[used_sensors]
test_df = test_df[used_sensors]

# Processed data - Numpy
train_full = train_full_df.values
test = test_df.values
train_labels_full = train_labels_full_df.values.squeeze()
test_labels = test_labels_df.values.squeeze()

In [ ]:
joined_train_rul = train_full_df.copy()
joined_train_rul['RUL'] = train_labels_full_df['RUL']
test_at_break_df = test_df.groupby('ID').last().reset_index()
test_at_break = test_at_break_df.values

train_labels_at_break = joined_train_rul.groupby('ID').last().reset_index()['RUL'].values
test_labels_at_break = test_labels_at_break_df.values[:, 0]

In [ ]:
train_groupby_full_df = train_full_df.groupby('ID', sort=False)
test_groupby_df = test_df.groupby('ID', sort=False)

train_labels_full_df['ID'] = joined_train_rul['ID']
train_labels_full_groupby_df = train_labels_full_df.groupby('ID', sort=False)

val_indices = np.random.choice(len(train_groupby_full_df), size = int(0.2 * len(train_groupby_full_df)))

val_arr = []
train_set_arr = []
val_labels_arr = []
train_set_labels_arr = []

for i in range(len(train_groupby_full_df)):
    if i in val_indices:
        val_arr.append(train_groupby_full_df.get_group(i+1))
        val_labels_arr.append(train_labels_full_groupby_df.get_group(i+1)['RUL'])
    else:
        train_set_arr.append(train_groupby_full_df.get_group(i+1))
        train_set_labels_arr.append(train_labels_full_groupby_df.get_group(i+1)['RUL'])

val_set_df = val_arr[0]
val_labels_df = val_labels_arr[0]
for i in range(1, len(val_arr)):
    val_set_df = pd.concat([val_set_df, val_arr[i]])
    val_labels_df = pd.concat([val_labels_df, val_labels_arr[i]])

train_set_df = train_set_arr[0]
train_set_labels_df = train_set_labels_arr[0]
for i in range(1, len(train_set_arr)):
    train_set_df = pd.concat([train_set_df, train_set_arr[i]])
    train_set_labels_df = pd.concat([train_set_labels_df, train_set_labels_arr[i]])

train_set = train_set_df.values
train_set_labels = train_set_labels_df.values
val_set = val_set_df.values
val_labels = val_labels_df.values
val_labels = np.expand_dims(val_labels, axis = 1)
train_set_labels = np.expand_dims(train_set_labels, axis = 1)
train_labels_full = np.expand_dims(train_labels_full, axis = 1)

In [ ]:
print(train_full_df.shape, train_full.shape)
print(train_labels_full_df.shape, train_labels_full.shape)
print(train_set_df.shape, train_set.shape)
print(train_set_labels_df.shape, train_set_labels.shape)
print(val_set_df.shape, val_set.shape)
print(val_labels_df.shape, val_labels.shape)

### Windows extraction

In [ ]:
def get_windows(data_df, labels_df, window_length, mode = 'train'):

    if mode == 'train':

        labels_df['ID'] = data_df['ID']

        data_groupby = data_df.groupby('ID', sort=False)
        labels_groupby = labels_df.groupby('ID', sort=False)

        val_indices = np.random.choice(len(data_groupby), size = int(0.2 * len(data_groupby)))

        tr_data_eng_arr = []
        tr_labels_eng_arr = []

        val_data_eng_arr = []
        val_labels_eng_arr = []

        for i in range(len(data_groupby)):
            if i in val_indices:
                val_data_eng_arr.append(data_groupby.get_group(i+1))
            else:
                tr_data_eng_arr.append(data_groupby.get_group(i+1))

        for i in range(len(labels_groupby)):
            if i in val_indices:
                val_labels_eng_arr.append(labels_groupby.get_group(i+1))
            else:
                tr_labels_eng_arr.append(labels_groupby.get_group(i+1))

        tr_data_windows = []
        tr_label_windows = []
        for index in range(len(tr_data_eng_arr)):
            tr_data_arr = tr_data_eng_arr[index].to_numpy()
            tr_labels_arr = tr_labels_eng_arr[index].to_numpy()
            for t in range(tr_data_arr.shape[0] - window_length + 1):
                tr_data_windows.append(tr_data_arr[t:t+window_length, :])
                tr_label_windows.append(tr_labels_arr[t+window_length - 1, 0])

        val_data_windows = []
        val_label_windows = []
        for index in range(len(val_data_eng_arr)):
            val_data_arr = val_data_eng_arr[index].to_numpy()
            val_labels_arr = val_labels_eng_arr[index].to_numpy()
            for t in range(val_data_arr.shape[0] - window_length + 1):
                val_data_windows.append(val_data_arr[t:t+window_length, :])
                val_label_windows.append(val_labels_arr[t+window_length - 1, 0])

        return np.array(tr_data_windows), np.array(tr_label_windows), np.array(val_data_windows), np.array(val_label_windows)

    else:

        labels_df['ID'] = data_df['ID']

        data_groupby = data_df.groupby('ID', sort=False)
        labels_groupby = labels_df.groupby('ID', sort=False)
        data_eng_arr = []
        labels_eng_arr = []

        for i in range(len(data_groupby)):
            data_eng_arr.append(data_groupby.get_group(i+1))

        for i in range(len(labels_groupby)):
            labels_eng_arr.append(labels_groupby.get_group(i+1))

        data_windows = []
        label_windows = []
        for index in range(len(data_eng_arr)):
            data_arr = data_eng_arr[index].to_numpy()
            labels_arr = labels_eng_arr[index].to_numpy()
            data_windows.append(data_arr[-window_length:, :])
            label_windows.append(labels_arr[-1, 0])

        return np.array(data_windows), np.array(label_windows)

## Random Forest Regressor

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
rf_param_grid = {
    'bootstrap': [True, False], 
    'max_depth': [6, 7, 8, 9, 10], 
    'min_samples_leaf': [30, 35, 40, 45, 50],
    'max_features': ['auto', 'log2', 'sqrt'], 
    'n_estimators': [100 * x for x in range(5, 11)],
    }

In [ ]:
rf = RandomForestRegressor(random_state=42)
rand_search = RandomizedSearchCV(estimator = rf, param_distributions = rf_param_grid, cv = 3, n_jobs = 1, verbose = 3, return_train_score=True)
rand_search.fit(train_full, train_labels_full)

In [ ]:
predictions_rf = rand_search.predict(test_at_break).round()
print(rand_search.best_params_)
rmse = np.sqrt(mean_squared_error(test_labels_at_break, predictions_rf))
print("RMSE: " + str(rmse))  

## Gradient Boosting Regressor

In [ ]:
gb = GradientBoostingRegressor()
gb.fit(train_full, train_labels_full)  

In [ ]:
predictions_gb = gb.predict(test_at_break).round()

rmse = np.sqrt(mean_squared_error(test_labels_at_break, predictions_gb))
print("RMSE: " + str(rmse))  

## Support Vector Regressor

In [ ]:
svmr = SVR(C=1.0, epsilon=0.2)
svmr.fit(train_full, train_labels_full)

In [ ]:
predictions_svmr = svmr.predict(test_at_break).round()
rmse = np.sqrt(mean_squared_error(test_labels_at_break, predictions_svmr))
print("RMSE: " + str(rmse))  

## Multi Layer Perceptron - Neural Network

In [ ]:
mlp_model = Sequential()
mlp_model.add(Dense(32, activation = 'relu', input_dim = train_set.shape[1]))
mlp_model.add(Dense(64, activation = 'relu'))
mlp_model.add(Dense(128 , activation = 'relu'))
mlp_model.add(Dropout(0.1))
mlp_model.add(Dense(1, activation = 'relu'))


mlp_model.compile(loss='mean_squared_error', optimizer='adam')
mlp_model.summary()
mlp_model.save_weights('mlp_weights.h5')

In [ ]:
mlp_model.compile(loss='mean_squared_error', optimizer='adam')
mlp_model.load_weights('mlp_weights.h5') 
mlp_history = mlp_model.fit(train_set, train_set_labels, epochs=100, validation_data = (val_set, val_labels), batch_size = 128)

In [ ]:
# PLOT TRAIN AND VALIDATION LOSS
def plot_loss(fit_history):
    plt.figure(figsize=(13,5))
    plt.plot(range(1, len(fit_history.history['loss'])+1), fit_history.history['loss'], label='train')
    plt.plot(range(1, len(fit_history.history['val_loss'])+1), fit_history.history['val_loss'], label='validate')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_loss(mlp_history)

In [ ]:
# TESTING FUNCTION
def evaluate(actual, pred, mode = 'test'):
    mse = mean_squared_error(actual, pred)
    rmse = np.sqrt(mse)
    variance = r2_score(actual, pred)
    print(mode + ' set RMSE: ' + str(rmse) + ', R2: ' + str(variance))

In [ ]:
# TESTING
train_full_pred = mlp_model.predict(train_full)
evaluate(train_labels_full, train_full_pred, 'train')

test_at_break_pred = mlp_model.predict(test_at_break)
evaluate(test_labels_at_break, test_at_break_pred)

## Convolutional Neural Network

In [ ]:
window_length = 20
cnn_tr_data, cnn_tr_labels, cnn_val_data, cnn_val_labels = get_windows(train_full_df, train_labels_full_df, window_length, mode='train')
cnn_test_data, cnn_test_labels = get_windows(test_df, test_labels_df, 20, mode = 'test')

cnn_tr_labels = np.expand_dims(cnn_tr_labels, axis=1)
cnn_val_labels = np.expand_dims(cnn_val_labels, axis=1)
cnn_test_labels = np.expand_dims(cnn_test_labels, axis=1)

### Model

In [ ]:
cnn_model = Sequential()
cnn_model.add(Convolution1D(256, 3, input_shape = (window_length, cnn_tr_data.shape[2])))
cnn_model.add(Convolution1D(128, 3, activation = 'relu'))
cnn_model.add(Convolution1D(64, 3, activation = 'relu'))
cnn_model.add(GlobalAveragePooling1D(data_format = 'channels_last', keepdims = False))
cnn_model.add(Dense(1, activation = 'relu'))

cnn_model.compile(loss='mean_squared_error', optimizer='adam')
cnn_model.save_weights('simple_lstm_weights.h5')

cnn_model.compile(loss='mean_squared_error', optimizer='adam')  
cnn_model.load_weights('simple_lstm_weights.h5')  

history = cnn_model.fit(cnn_tr_data, cnn_tr_labels,
                        validation_data=(cnn_val_data, cnn_val_labels),
                        epochs=50,
                        batch_size=128)

In [ ]:
# TESTING FUNCTION
def evaluate(actual, pred, mode = 'test'):
    mse = mean_squared_error(actual, pred)
    rmse = np.sqrt(mse)
    variance = r2_score(actual, pred)
    print(mode + ' set RMSE: ' + str(rmse) + ', R2: ' + str(variance))

In [ ]:
# TESTING
train_cnn_pred = cnn_model.predict(cnn_tr_data)
evaluate(cnn_tr_labels, train_cnn_pred, 'train')

test_cnn_pred = cnn_model.predict(cnn_test_data)
evaluate(cnn_test_labels, test_cnn_pred)

## LSTM Neural Network

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(32, activation='tanh', input_shape=(window_length, cnn_tr_data.shape[2])))
lstm_model.add(Dense(1))

lstm_model.compile(loss='mean_squared_error', optimizer='adam')
lstm_model.save_weights('simple_lstm_weights.h5')

lstm_model.compile(loss='mean_squared_error', optimizer='adam')  
lstm_model.load_weights('simple_lstm_weights.h5')  

history = lstm_model.fit(cnn_tr_data, cnn_tr_labels,
                        validation_data=(cnn_val_data, cnn_val_labels),
                        epochs=50,
                        batch_size=128)

In [ ]:
# PLOT LOSS HISTORY
def plot_loss(fit_history):
    plt.figure(figsize=(13,5))
    plt.plot(range(1, len(fit_history.history['loss'])+1), fit_history.history['loss'], label='train')
    plt.plot(range(1, len(fit_history.history['val_loss'])+1), fit_history.history['val_loss'], label='validate')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_loss(history)

In [ ]:
# TESTING FUNCTION
def evaluate(actual, pred, mode = 'test'):
    mse = mean_squared_error(actual, pred)
    rmse = np.sqrt(mse)
    variance = r2_score(actual, pred)
    print(mode + ' set RMSE: ' + str(rmse) + ', R2: ' + str(variance))

In [ ]:
# TESTING
train_cnn_pred = lstm_model.predict(cnn_tr_data)
evaluate(cnn_tr_labels, train_cnn_pred, 'train')

test_cnn_pred = lstm_model.predict(cnn_test_data)
evaluate(cnn_test_labels, test_cnn_pred)